In [28]:
import numpy as np
import pandas as pd
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator



In [35]:
# NOTE: Make sure that the class is labeled 'target' in the data file
df = pd.read_csv('scaled_df_on_50%features.csv')
target = df.fraud.values
df = df.drop(['fraud','adsh'], axis=1).values

In [36]:
X_train, X_test, y_train, y_test = train_test_split(df, target)


In [37]:
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=False, criterion="gini", max_features=0.9000000000000001, min_samples_leaf=2, min_samples_split=5, n_estimators=100)),
    GradientBoostingClassifier(learning_rate=0.5, max_depth=6, max_features=0.1, min_samples_leaf=1, min_samples_split=13, n_estimators=100, subsample=0.6000000000000001)
)

exported_pipeline.fit(X_train, y_train)



Pipeline(memory=None,
     steps=[('stackingestimator', StackingEstimator(estimator=ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features=0.9000000000000001,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_l....6000000000000001, tol=0.0001,
              validation_fraction=0.1, verbose=0, warm_start=False))])

In [9]:
results = exported_pipeline.predict(testing_features)

In [11]:
clf_train_preds = exported_pipeline.predict(training_features)
clf_test_preds = results

In [13]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds)
    print("Model: {}".format(model_name))
    print("Accuracy: {}".format(acc))
    print("F1-Score: {}".format(f1))
    
print("Training Metrics")
# display_acc_and_f1_score(y_train, adaboost_train_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(training_target, clf_train_preds, model_name='Stacked Classifier')
print("")
print("Testing Metrics")
# display_acc_and_f1_score(y_test, adaboost_test_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(testing_target, clf_test_preds, model_name='Stacked Classifier')


Training Metrics

Model: Stacked Classifier
Accuracy: 1.0
F1-Score: 1.0

Testing Metrics

Model: Stacked Classifier
Accuracy: 0.9969558599695586
F1-Score: 0.9969742813918305


In [14]:
clf_confusion_matrix = confusion_matrix(testing_target, clf_test_preds)
clf_confusion_matrix

array([[1302,    8],
       [   0, 1318]])

In [17]:
clf_classification_report = classification_report(testing_target, clf_test_preds)
print(clf_classification_report)

              precision    recall  f1-score   support

         0.0       1.00      0.99      1.00      1310
         1.0       0.99      1.00      1.00      1318

   micro avg       1.00      1.00      1.00      2628
   macro avg       1.00      1.00      1.00      2628
weighted avg       1.00      1.00      1.00      2628



In [38]:
from sklearn.metrics import f1_score
f1_score(y_test, clf_test_preds)

0.5119881962375508

In [24]:
from sklearn.model_selection import train_test_split, cross_val_score
print('Mean Stacked Classifier Cross-Val Score (k=5):')
print(cross_val_score(exported_pipeline, features, target, cv=5).mean())

Mean Stacked Classifier Cross-Val Score (k=5):
0.5433974845929588


In [25]:
import matplotlib.pyplot as plt

def plot_feature_importances(model):
    n_features = X_train.shape[1]
    plt.figure(figsize=(8,8))
    plt.barh(range(n_features), gbt_clf.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), X_train.columns.values) 
    plt.xlabel("Feature importance")
    plt.ylabel("Feature")

plot_feature_importances(exported_pipeline)

NameError: name 'X_train' is not defined